In [38]:
import sys
sys.path.append('../src')

import os
import requests
import json
import pandas as pd
import betfairlightweight
from betfairlightweight import filters
from datetime import datetime, timedelta
import time

from betfair import Betfair

pd.set_option('display.max_columns', None)

In [39]:
app_key = 'ABGJLOlKaLtTsMIp'

username1 = input("Email:")
password1 = input("Password:")

In [40]:
trading = betfairlightweight.APIClient(
        username1, password1, app_key
    )
trading.login_interactive()

trading.session_token

trading.keep_alive()

<KeepAliveResource>

In [41]:
# Set the start and end time to cover the entire day in UTC
today = datetime.utcnow()
start_time = today.replace(hour=0, minute=0, second=0, microsecond=0)
end_time = today.replace(hour=23, minute=59, second=59, microsecond=999999)

In [42]:
# Define filter for horse racing markets
market_filter = betfairlightweight.filters.market_filter(
    event_type_ids=['7'],
    market_countries=['GB'],
    market_type_codes=['WIN'], 
    market_start_time={'from': start_time.isoformat(),
                       'to' : end_time.isoformat()}
)

# Request market catalogue for all horse races on the date
results = trading.betting.list_market_catalogue(
    filter=market_filter,
    market_projection=["RUNNER_DESCRIPTION", 
            "RUNNER_METADATA", 
            "COMPETITION", 
            "EVENT", 
            "EVENT_TYPE", 
            "MARKET_DESCRIPTION", 
            "MARKET_START_TIME",],  # Include runner metadata
    max_results=1000  # Adjust based on expected number of races
)

In [43]:
for i in results:
        print(
            "{0} {1:d}:{2:02d} {3} ({4})".format(
                i.market_id,
                i.market_start_time.hour, 
                i.market_start_time.minute, 
                i.event.venue,  
                i.description.market_type,
            )
        )


1.233233389 14:00 Ayr (WIN)
1.233233029 14:05 Chester (WIN)
1.233233172 14:15 Newbury (WIN)
1.233232294 14:20 Newmarket (WIN)
1.233233396 14:35 Ayr (WIN)
1.233233035 14:40 Chester (WIN)
1.233233179 14:50 Newbury (WIN)
1.233232300 14:55 Newmarket (WIN)
1.233233405 15:10 Ayr (WIN)
1.233233042 15:15 Chester (WIN)
1.233233285 15:19 Wolverhampton (WIN)
1.233233185 15:25 Newbury (WIN)
1.233232306 15:30 Newmarket (WIN)
1.233233412 15:40 Ayr (WIN)
1.233233049 15:45 Chester (WIN)
1.233233290 15:50 Wolverhampton (WIN)
1.233233217 16:00 Newbury (WIN)
1.233232313 16:05 Newmarket (WIN)
1.233233419 16:15 Ayr (WIN)
1.233233056 16:20 Chester (WIN)
1.233233297 16:25 Wolverhampton (WIN)
1.233233244 16:35 Newbury (WIN)
1.233233304 16:55 Wolverhampton (WIN)
1.233233311 17:30 Wolverhampton (WIN)
1.233233318 18:00 Wolverhampton (WIN)
1.233233325 18:30 Wolverhampton (WIN)
1.233233332 19:00 Wolverhampton (WIN)
1.233233338 19:30 Wolverhampton (WIN)


In [44]:
import pandas as pd

# Initialize an empty list to store market and runner details
data = []

# Iterate over each market in the market catalogue
for market in results:
    market_id = market.market_id
    market_name = market.market_name
    market_start_time = market.market_start_time
    market_type = market.description.market_type

    # Extract the location/venue of the race
    event = market.event
    race_location = event.venue if event else "Unknown Location"
    
    # Iterate over each runner in the market
    for runner in market.runners:
        runner_id = runner.selection_id
        runner_name = runner.runner_name
        
        # Append a dictionary with all relevant data to the list
        data.append({
            'market_id': market_id,
            'market_start_time': market_start_time,
            'venue': race_location,
            'market_type': market_type,
            'selection_id': runner_id,
            'horse_name': runner_name,
        })

# Convert the list of dictionaries to a pandas DataFrame
df1 = pd.DataFrame(data)

# Display the DataFrame
print(df1)


       market_id   market_start_time          venue market_type  selection_id  \
0    1.233233389 2024-09-21 14:00:00            Ayr         WIN      73153311   
1    1.233233389 2024-09-21 14:00:00            Ayr         WIN      69727304   
2    1.233233389 2024-09-21 14:00:00            Ayr         WIN       9978944   
3    1.233233389 2024-09-21 14:00:00            Ayr         WIN      12419804   
4    1.233233389 2024-09-21 14:00:00            Ayr         WIN      68134685   
..           ...                 ...            ...         ...           ...   
316  1.233233338 2024-09-21 19:30:00  Wolverhampton         WIN       1366807   
317  1.233233338 2024-09-21 19:30:00  Wolverhampton         WIN      55953212   
318  1.233233338 2024-09-21 19:30:00  Wolverhampton         WIN       5176861   
319  1.233233338 2024-09-21 19:30:00  Wolverhampton         WIN      65217691   
320  1.233233338 2024-09-21 19:30:00  Wolverhampton         WIN      50747818   

           horse_name  
0  

# I need to get the a list of id's from the market_id column and then create a loop for the list_market_book 

In [45]:
market_id_list = df1['market_id'].drop_duplicates().to_list()
len(market_id_list)

28

In [46]:
# Define a delay between requests
delay = 0.05  # Delay in seconds

# Loop through market IDs and fetch data with a delay
market_books = []
for market_id in market_id_list:
    market_book = trading.betting.list_market_book(
        market_ids=[market_id],
        price_projection=betfairlightweight.filters.price_projection(
            price_data=betfairlightweight.filters.price_data(ex_all_offers=True)
        ),
    )
    market_books.append(market_book)
    
    # Print or process the market_book as needed
    print(market_book)
    
    # Sleep to avoid overloading the API
    time.sleep(delay)

[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]


In [47]:
# Example: if market_books is a list of lists of MarketBook objects
data = []

# Iterate over each list of MarketBook objects
for book_list in market_books:
    # Iterate over each MarketBook object in the current list
    for market_book in book_list:
        market_id = market_book.market_id
        status = market_book.status
        total_matched = market_book.total_matched

        # Iterate over each runner in the market
        for runner in market_book.runners:
            runner_id = runner.selection_id
            runner_status = runner.status
            last_price_traded = runner.last_price_traded
            runner_total_matched = runner.total_matched
            
            # Append a dictionary with all relevant data to the list
            data.append({
                'market_id': market_id,
                'status': status,
                'total_matched': total_matched,
                'selection_id': runner_id,
                'status': runner_status,
                'morning_price': last_price_traded,
            })



# Convert the list of dictionaries into a DataFrame
df2 = pd.DataFrame(data)

df2 = df2[['market_id', 'selection_id', 'status', 'morning_price', 'total_matched']]
df2

,market_id,selection_id,status,morning_price,total_matched
0,1.233233389,73153311,ACTIVE,5.2,147955.54
1,1.233233389,69727304,ACTIVE,8.6,147955.54
2,1.233233389,9978944,ACTIVE,8.2,147955.54
3,1.233233389,12419804,ACTIVE,7.2,147955.54
4,1.233233389,68134685,ACTIVE,8.2,147955.54
...,...,...,...,...,...
316,1.233233338,1366807,ACTIVE,19.5,4537.80
317,1.233233338,55953212,ACTIVE,38.0,4537.80
318,1.233233338,5176861,ACTIVE,50.0,4537.80
319,1.233233338,65217691,ACTIVE,80.0,4537.80


# join df1 and df2

In [48]:
df3 = df1.merge(df2, on = ['market_id', 'selection_id'], how = 'left')

df3['market_id'] = df3['market_id'].astype(str)


current_date = datetime.now().strftime('%Y-%m-%d')
file_name = f"{current_date}_data.csv"


df3.to_csv('../data/daily/' + file_name, index=False)
df3

,market_id,market_start_time,venue,market_type,selection_id,horse_name,status,morning_price,total_matched
0,1.233233389,2024-09-21 14:00:00,Ayr,WIN,73153311,Sky Majesty,ACTIVE,5.2,147955.54
1,1.233233389,2024-09-21 14:00:00,Ayr,WIN,69727304,Grande Marques,ACTIVE,8.6,147955.54
2,1.233233389,2024-09-21 14:00:00,Ayr,WIN,9978944,Town And Country,ACTIVE,8.2,147955.54
3,1.233233389,2024-09-21 14:00:00,Ayr,WIN,12419804,Stormy Impact,ACTIVE,7.2,147955.54
4,1.233233389,2024-09-21 14:00:00,Ayr,WIN,68134685,Englemere,ACTIVE,8.2,147955.54
...,...,...,...,...,...,...,...,...,...
316,1.233233338,2024-09-21 19:30:00,Wolverhampton,WIN,1366807,Sexy Rexy,ACTIVE,19.5,4537.80
317,1.233233338,2024-09-21 19:30:00,Wolverhampton,WIN,55953212,Aquifolia,ACTIVE,38.0,4537.80
318,1.233233338,2024-09-21 19:30:00,Wolverhampton,WIN,5176861,Renoir,ACTIVE,50.0,4537.80
319,1.233233338,2024-09-21 19:30:00,Wolverhampton,WIN,65217691,Exceptionality,ACTIVE,80.0,4537.80
